# Fine-Tuning Phi-4 with PEFT (LoRA)

This notebook shows how to fine-tune the `OpenAI/phi-4` model from Hugging Face using the PEFT library with LoRA adapters.

## 1. Install Dependencies

In [ ]:
# !pip install -U transformers datasets accelerate peft bitsandbytes

## 2. Imports and Configuration

In [ ]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import LoraConfig, get_peft_model, TaskType

# Configuration
MODEL_NAME = "microsoft/phi-4"
OUTPUT_DIR = "./phi4_finetuned"
BATCH_SIZE = 8
NUM_EPOCHS = 3
LEARNING_RATE = 2e-4
MAX_SEQ_LENGTH = 256

# LoRA configuration
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)


## 3. Load Tokenizer and Dataset

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load a sample instruction dataset
dataset = load_dataset("yahma/alpaca-cleaned")
dataset

## 4. Preprocess & Tokenize

In [ ]:
def preprocess_fn(examples):
    inputs = ["### Instruction:\n" + ins + "\n### Response:\n" for ins in examples["instruction"]]
    model_inputs = tokenizer(inputs, max_length=MAX_SEQ_LENGTH, truncation=True, padding="max_length")
    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=MAX_SEQ_LENGTH, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized = dataset.map(
    preprocess_fn,
    batched=True,
    remove_columns=["instruction", "input", "output"]
)
tokenized

## 5. Prepare Model with LoRA Adapters

In [ ]:
# Load base model in 8-bit for memory efficiency
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # load_in_8bit=True,
    device_map="auto"
)

# Set up LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT
)
model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()

## 6. Data Collator & Training Arguments

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
)

## 7. Initialize Trainer and Train

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized.get("test"),
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

## 8. Save Fine-Tuned Model and Adapters

In [ ]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model and adapters saved to {OUTPUT_DIR}")

## 9. Inference Example
```python
from transformers import pipeline
generator = pipeline(
    'text2text-generation',
    model=OUTPUT_DIR,
    tokenizer=tokenizer,
    device=0
)
print(generator('Tell me more about generative AI', max_length=150, do_sample=True))
```